# clean pmlb results for mock experiment

In [3]:

import pandas as pd
import numpy as np

data = pd.read_csv('sklearn-benchmark5-data-edited.tsv.gz',                          
                         compression='gzip', sep='\t',                                           
                         names=['dataset',
                                 'classifier',
                                 'parameters',
                                 'accuracy', 
                                 'macrof1',
                                 'bal_accuracy']).fillna('')  

data['accuracy'] = data['accuracy'].apply(lambda x: round(x, 3))
print('loaded ',data['dataset'].unique().shape[0],'datasets and ', data['classifier'].unique().shape[0],'classifiers')
# subset data to classifiers used in PennAI
pennai_classifiers = ['LogisticRegression', 'RandomForestClassifier', 'SVC', 
                      'KNeighborsClassifier', 'DecisionTreeClassifier', 'GradientBoostingClassifier']
mask = np.array([c in pennai_classifiers for c in data['classifier'].values])
data = data.loc[mask,:]
print('classifiers:',data['classifier'].unique())
for ml, df_g in data.groupby('classifier'):
    print('parameters for ',ml,'(',len(df_g['parameters'].unique()),'):',df_g['parameters'].unique())
# print(data[:10])
data.groupby('classifier').count()

/home/bill/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


loaded  166 datasets and  13 classifiers
classifiers: ['DecisionTreeClassifier' 'GradientBoostingClassifier'
 'KNeighborsClassifier' 'LogisticRegression' 'RandomForestClassifier'
 'SVC']
parameters for  DecisionTreeClassifier ( 154 ): ['min_weight_fraction_leaf=0.0,max_features=0.1,criterion=entropy'
 'min_weight_fraction_leaf=0.0,max_features=0.1,criterion=gini'
 'min_weight_fraction_leaf=0.0,max_features=0.25,criterion=entropy'
 'min_weight_fraction_leaf=0.0,max_features=0.25,criterion=gini'
 'min_weight_fraction_leaf=0.0,max_features=0.5,criterion=entropy'
 'min_weight_fraction_leaf=0.0,max_features=0.5,criterion=gini'
 'min_weight_fraction_leaf=0.0,max_features=0.75,criterion=entropy'
 'min_weight_fraction_leaf=0.0,max_features=0.75,criterion=gini'
 'min_weight_fraction_leaf=0.0,max_features=None,criterion=entropy'
 'min_weight_fraction_leaf=0.0,max_features=None,criterion=gini'
 'min_weight_fraction_leaf=0.0,max_features=log2,criterion=entropy'
 'min_weight_fraction_leaf=0.0,max_f

parameters for  GradientBoostingClassifier ( 6301 ): ['loss=deviance,learning_rate=0.01,n_estimators=10,max_depth=1,max_features=0.2'
 'loss=deviance,learning_rate=0.01,n_estimators=10,max_depth=1,max_features=0.3'
 'loss=deviance,learning_rate=0.01,n_estimators=10,max_depth=1,max_features=0.4'
 ...
 'loss=exponential,learning_rate=50.0,n_estimators=500,max_depth=None,max_features=0.9'
 'loss=exponential,learning_rate=50.0,n_estimators=500,max_depth=None,max_features=log2'
 'loss=exponential,learning_rate=50.0,n_estimators=500,max_depth=None,max_features=sqrt']
parameters for  KNeighborsClassifier ( 54 ): ['n_neighbors=1,weights=distance' 'n_neighbors=1,weights=uniform'
 'n_neighbors=10,weights=distance' 'n_neighbors=10,weights=uniform'
 'n_neighbors=100,weights=distance' 'n_neighbors=100,weights=uniform'
 'n_neighbors=11,weights=distance' 'n_neighbors=11,weights=uniform'
 'n_neighbors=12,weights=distance' 'n_neighbors=12,weights=uniform'
 'n_neighbors=13,weights=distance' 'n_neighbors

,dataset,parameters,accuracy,macrof1,bal_accuracy
classifier,,,,,
DecisionTreeClassifier,25410,25410,25410,25410,25410
GradientBoostingClassifier,769900,769900,769900,769900,769900
KNeighborsClassifier,8831,8831,8831,8831,8831
LogisticRegression,39145,39145,39145,39145,39145
RandomForestClassifier,126602,126602,126602,126602,126602
SVC,143179,143179,143179,143179,143179


In [42]:
#datasets to remove:
big_datasets = ['poker', 'kddcup', 'sleep', 'fars', 'mnist', 'connect-4', 'shuttle', 'adult', 'krkopt', 
                'letter', 'magic', 'nursery', 'pendigits', 'coil2000', 'agaricus-lepiota','optdigits']
mask = np.array([d not in big_datasets for d in data['dataset'].values])
clean_data = data.loc[mask,:]
clean_data.groupby('dataset').count()

# remove alg-params that don't cover all datasets
clean_data.loc[:, 'algorithm-parameters'] = (                                             
                                       clean_data['algorithm'].values + '|' +                     
                                       clean_data['parameters'].values) 
all_datasets = np.unique(clean_data['dataset'].values)
algp_toremove = []
for algp, group in clean_data.groupby('algorithm-parameters'):
    if (group['dataset'].count()<150):
        #print(algp, '\n\t is missing results for', [d for d in all_datasets if d not in np.unique(group['dataset'])])
        #print('removing',algp)
        algp_toremove.append(algp)
        
mask = np.array([ap not in algp_toremove for ap in clean_data['algorithm-parameters'].values])
clean_data = clean_data.loc[mask,:]
for alg, grp in clean_data.groupby('algorithm'):
    assert (len(grp)==3423)

#clean_data.groupby('algorithm-parameters').count()
clean_data.drop('algorithm-parameters',axis=1,inplace=True)
clean_data.to_csv('sklearn-benchmark5-data-mock_experiment.tsv.gz',compression='gzip',index=False,sep='\t',
                  header=['dataset',
                                 'algorithm',
                                 'parameters',
                                 'accuracy', 
                                 'macrof1',
                                 'bal_accuracy'])


/home/bill/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/bill/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [44]:
clean_data.groupby('dataset').count()

,algorithm,parameters,accuracy,macrof1,bal_accuracy
dataset,,,,,
GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_EDM-1_1,3423,3423,3423,3423,3423
GAMETES_Epistasis_2-Way_20atts_0.1H_EDM-1_1,3423,3423,3423,3423,3423
GAMETES_Epistasis_2-Way_20atts_0.4H_EDM-1_1,3423,3423,3423,3423,3423
GAMETES_Epistasis_3-Way_20atts_0.2H_EDM-1_1,3423,3423,3423,3423,3423
GAMETES_Heterogeneity_20atts_1600_Het_0.4_0.2_50_EDM-2_001,3423,3423,3423,3423,3423
GAMETES_Heterogeneity_20atts_1600_Het_0.4_0.2_75_EDM-2_001,3423,3423,3423,3423,3423
Hill_Valley_with_noise,3423,3423,3423,3423,3423
Hill_Valley_without_noise,3423,3423,3423,3423,3423
allbp,3423,3423,3423,3423,3423


In [24]:
import pandas as pd
import numpy as np
data = pd.read_csv('sklearn-benchmark5-data-mock_experiment.tsv.gz',compression='gzip', sep='\t').fillna('')  
data.groupby('algorithm').count()
# reduce GBC
n_est = ['n_estimators='+str(n) for n in [100,500,1000]]
max_depth = ['max_depth='+str(d) for d in [2,4,6,10]]
max_features = ['max_features='+str(f) for f in ['sqrt','log2',0.9]]

gbc = data['algorithm']=='GradientBoostingClassifier'
mask = [False for g in gbc]
for n in n_est:
    mask = mask | (gbc & np.array([n in p for p in data['parameters'].values]))
mask = (mask | ~gbc)
data = data.loc[mask,:]

gbc = data['algorithm']=='GradientBoostingClassifier'
mask = [False for g in gbc]
for n in max_depth:
    mask = mask | (gbc & np.array([n in p for p in data['parameters'].values]))
mask = (mask | ~gbc)
data = data.loc[mask,:]

gbc = data['algorithm']=='GradientBoostingClassifier'
mask = [False for g in gbc]
for n in max_features:
    mask = mask | (gbc & np.array([n in p for p in data['parameters'].values]))
mask = (mask | ~gbc)
data = data.loc[mask,:]


# reduce Random Forest n_estimators

rfc = data['algorithm']=='RandomForestClassifier'
mask = [False for g in rfc]
for n in n_est:
    mask = mask | (rfc & np.array([n in p for p in data['parameters'].values]))
mask = (mask | ~rfc)
data = data.loc[mask,:]

max_features = ['max_features='+str(f) for f in ['sqrt','log2',None]]
rfc = data['algorithm']=='RandomForestClassifier'
mask = [False for g in rfc]
for n in max_features:
    mask = mask | (rfc & np.array([n in p for p in data['parameters'].values]))
mask = (mask | ~rfc)
data = data.loc[mask,:]

data.groupby('algorithm').count()

# write to file 
data.to_csv('sklearn-benchmark5-data-mock_experiment.tsv.gz',compression='gzip',index=False,sep='\t',
                  header=['dataset',
                                 'algorithm',
                                 'parameters',
                                 'accuracy', 
                                 'macrof1',
                                 'bal_accuracy'])

In [21]:
for d in data.loc[data['algorithm']=='RandomForestClassifier']['parameters'].unique():
    print(d)

n_estimators=10,min_weight_fraction_leaf=0.0,max_features=0.1,criterion=entropy
n_estimators=10,min_weight_fraction_leaf=0.0,max_features=0.1,criterion=gini
n_estimators=10,min_weight_fraction_leaf=0.0,max_features=0.25,criterion=entropy
n_estimators=10,min_weight_fraction_leaf=0.0,max_features=0.25,criterion=gini
n_estimators=10,min_weight_fraction_leaf=0.0,max_features=0.5,criterion=entropy
n_estimators=10,min_weight_fraction_leaf=0.0,max_features=0.5,criterion=gini
n_estimators=10,min_weight_fraction_leaf=0.0,max_features=0.75,criterion=entropy
n_estimators=10,min_weight_fraction_leaf=0.0,max_features=0.75,criterion=gini
n_estimators=10,min_weight_fraction_leaf=0.0,max_features=None,criterion=entropy
n_estimators=10,min_weight_fraction_leaf=0.0,max_features=None,criterion=gini
n_estimators=10,min_weight_fraction_leaf=0.0,max_features=log2,criterion=entropy
n_estimators=10,min_weight_fraction_leaf=0.0,max_features=log2,criterion=gini
n_estimators=10,min_weight_fraction_leaf=0.0,max_f